In [0]:
import os

In [0]:
!git clone https://github.com/h2oai/pystacknet

Cloning into 'pystacknet'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), done.


In [0]:
os.chdir('./pystacknet')

In [0]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating pystacknet.egg-info
writing pystacknet.egg-info/PKG-INFO
writing dependency_links to pystacknet.egg-info/dependency_links.txt
writing requirements to pystacknet.egg-info/requires.txt
writing top-level names to pystacknet.egg-info/top_level.txt
writing manifest file 'pystacknet.egg-info/SOURCES.txt'
writing manifest file 'pystacknet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pystacknet
copying pystacknet/pystacknet.py -> build/lib/pystacknet
copying pystacknet/__init__.py -> build/lib/pystacknet
copying pystacknet/metrics.py -> build/lib/pystacknet
creating build/lib/pystacknet/test
copying pystacknet/test/__init__.py -> build/lib/pystacknet/test
copying pystacknet/test/test_amazon.py -> build/lib/pystacknet/test
copying pystacknet/test/test_pystacknet.py -> build/lib/pystacknet/test
creating build/bdist.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pystacknet.pystacknet import StackNetClassifier
import numpy as np
from pystacknet.pystacknet import StackNetClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn import preprocessing
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier,ExtraTreesRegressor

In [0]:
os.chdir('/content')

In [0]:
# setup kaggle, so that you can get the dataset directly 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle competitions download -c santander-customer-transaction-prediction

 99% 121M/122M [00:00<00:00, 123MB/s]
100% 122M/122M [00:00<00:00, 157MB/s]
  0% 0.00/463k [00:00<?, ?B/s]
100% 463k/463k [00:00<00:00, 49.6MB/s]
 88% 107M/122M [00:00<00:00, 102MB/s]  
100% 122M/122M [00:01<00:00, 119MB/s]


In [0]:
!unzip test.csv.zip 
!unzip train.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [0]:
idx = features = train.columns.values[2:202]
for df in [test,train]:
    df['sum'] = df[idx].sum(axis=1)  
    df['min'] = df[idx].min(axis=1)
    df['max'] = df[idx].max(axis=1)
    df['mean'] = df[idx].mean(axis=1)
    df['std'] = df[idx].std(axis=1)
    df['skew'] = df[idx].skew(axis=1)
    df['kurt'] = df[idx].kurtosis(axis=1)
    df['med'] = df[idx].median(axis=1)


In [0]:
train_copy=train.copy()
test_copy=test.copy()

In [0]:
features = [c for c in train_copy.columns if c not in ['ID_code', 'target']]
for feature in features:
    train_copy['r2_'+feature] = np.round(train_copy[feature], 2)
    test_copy['r2_'+feature] = np.round(test_copy[feature], 2)
    train_copy['r1_'+feature] = np.round(train_copy[feature], 1)
    test_copy['r1_'+feature] = np.round(test_copy[feature], 1)

In [0]:
X=train_copy.drop(columns=['ID_code','target']).values
y=train_copy.target.values
X_test=test_copy.drop(columns=['ID_code']).values

In [0]:
 models=[ 
            ######## First level ########
            [#RandomForestClassifier (n_estimators=200, criterion="entropy", max_depth=3, max_features=0.3, random_state=1,n_jobs=1),
             #ExtraTreesClassifier (n_estimators=200, criterion="entropy", max_depth=3, max_features=0.3, random_state=1,n_jobs=1),
             #ExtraTreesRegressor (n_estimators=100, max_depth=5, max_features=0.3, random_state=1,n_jobs=1),
             GradientBoostingClassifier(n_estimators=310,max_depth=5, max_features=0.5, random_state=1),
             LogisticRegression(random_state=1),
             XGBClassifier(max_depth=5, n_estimators=300, objective="binary:logistic", booster="gbtree", random_state=1,n_jobs=1 ),
             LGBMClassifier(boosting_type='gbdt', num_leaves=10, max_depth= 6, learning_rate=0.01, n_estimators=300,objective="binary",metric='auc',min_child_samples=10, subsample=0.9, subsample_freq=1, colsample_bytree=0.5,random_state=1, n_jobs=1)             

             ],
            ######## Second level ########
            [RandomForestClassifier (n_estimators=300, criterion="entropy", max_depth=5, max_features=0.7, random_state=1)]
            ]

In [0]:
model=StackNetClassifier(models, metric="auc", folds=3, restacking=True,
                         use_retraining=True, use_proba=True, random_state=2019,
                         n_jobs=1, verbose=1)

In [0]:
model.fit(X,y )

====================== Start of Level 0 ======================
Input Dimensionality 624 at Level 0 
4 models included in Level 0 


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fold 1/3 , model 0 , auc===0.878276 
Fold 1/3 , model 1 , auc===0.863501 
Fold 1/3 , model 2 , auc===0.879268 
Fold 1/3 , model 3 , auc===0.803974 
=========== end of fold 1 in level 0 ===========


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fold 2/3 , model 0 , auc===0.880347 
Fold 2/3 , model 1 , auc===0.868142 
Fold 2/3 , model 2 , auc===0.881669 
Fold 2/3 , model 3 , auc===0.808664 
=========== end of fold 2 in level 0 ===========


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Fold 3/3 , model 0 , auc===0.876615 
Fold 3/3 , model 1 , auc===0.863809 
Fold 3/3 , model 2 , auc===0.877583 
Fold 3/3 , model 3 , auc===0.808174 
=========== end of fold 3 in level 0 ===========
Level 0, model 0 , auc===0.878413 
Level 0, model 1 , auc===0.865151 
Level 0, model 2 , auc===0.879507 
Level 0, model 3 , auc===0.806937 


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Output dimensionality of level 0 is 4 
====================== End of Level 0 ======================
 level 0 lasted 28819.425099 seconds 
====================== Start of Level 1 ======================
Input Dimensionality 628 at Level 1 
1 models included in Level 1 


In [0]:
preds=model.predict_proba(X_test)[:,1]

In [0]:
test_copy['target']=preds

In [0]:
submission=test_copy[['ID_code','target']]
submission.head()

In [0]:
submission.to_csv('submission.csv',index=False)

In [0]:
!kaggle competitions submit -c santander-customer-transaction-prediction -f submission.csv -m "Message"